In [1]:
from os.path import isfile
import pandas as pd
import gc
import re
import numpy as np
import scipy.stats
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pickle

pd.set_option('float_format', '{:.4f}'.format)

In [2]:
def_column_order = ['Price', 'Year', 'Mileage', 'City', 'State', 'Make', 'Model']
numeric_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']


def get_column_order(current_order: list):
    return [x for x in def_column_order if x in current_order]


def proceed_cat_str(in_cat: pd.Series, trim=-1):
    res_cat = in_cat.map(
        lambda x: "".join(re.findall("[a-zA-Z0-9]+", x)) if type(x) is str else np.NaN
    )

    res_cat = res_cat.map(
        lambda x: str.upper(x) if type(x) is str else np.NaN
    )

    if trim != -1:
        res_cat = res_cat.str[:trim]

    return res_cat


def concat_cat(src_df: pd.DataFrame, name: str):
    res_cat = pd.Series(dtype=str)
    res_cat.name = name

    for col in src_df:
        if res_cat.shape[0] == 0:
            res_cat = src_df[col].astype(str)
        else:
            res_cat = res_cat + src_df[col].astype(str)

    return res_cat

# **Анализ данных.**

Взглянем издалека на числовые данные.

In [3]:
src_df = pd.read_csv('./data/true_car_listings_prepeared.csv')
src_df.drop_duplicates(inplace=True)

numeric = src_df.select_dtypes(include=numeric_types)

print(numeric.describe())

            Price        Year      Mileage
count 852105.0000 852105.0000  731162.0000
mean   20121.2714   2013.2891   52519.6486
std    14123.2041      3.4150   42036.4440
min        0.0000   1997.0000       5.0000
25%    11997.0000   2012.0000   23837.0000
50%    17900.0000   2014.0000   40265.0000
75%    26398.0000   2016.0000   72228.0000
max   499500.0000   2018.0000 2856196.0000


Выбросы точно присутствуют. Попробую визуально.

![title](img/06.png)

Что и требовалось показать, что называется.

In [4]:
print(numeric.describe(percentiles=np.arange(start=0.0, stop=1.00, step=0.05)))

            Price        Year      Mileage
count 852105.0000 852105.0000  731162.0000
mean   20121.2714   2013.2891   52519.6486
std    14123.2041      3.4150   42036.4440
min        0.0000   1997.0000       5.0000
0%         0.0000   1997.0000       5.0000
5%         0.0000   2006.0000    6577.0000
10%     6550.0000   2008.0000   12126.0000
15%     8995.0000   2010.0000   16514.0000
20%    10850.0000   2011.0000   20251.0000
25%    11997.0000   2012.0000   23837.0000
30%    13277.0000   2013.0000   27300.0000
35%    14455.0000   2013.0000   30543.0000
40%    15500.0000   2014.0000   33685.0000
45%    16733.0000   2014.0000   36745.0000
50%    17900.0000   2014.0000   40265.0000
55%    18990.0000   2014.0000   43982.0000
60%    20295.0000   2015.0000   48539.0000
65%    21999.0000   2015.0000   54845.0000
70%    23998.0000   2015.0000   62988.0000
75%    26398.0000   2016.0000   72228.0000
80%    28900.0000   2016.0000   82443.0000
85%    31843.0000   2016.0000   94358.0000
90%    3559

Интересно. А сколько у нас записей до, скажем, 2000-го года?

In [5]:
print(numeric.where(src_df['Year'] <= 2000).describe())

            Price      Year     Mileage
count   4557.0000 4557.0000   3871.0000
mean    6905.0266 1998.9928 144933.2304
std    10903.3134    1.0571  69101.1093
min        0.0000 1997.0000    111.0000
25%     2995.0000 1998.0000  98420.5000
50%     4680.0000 1999.0000 140723.0000
75%     7500.0000 2000.0000 183427.0000
max   224995.0000 2000.0000 999999.0000


2010-го?

In [6]:
print(numeric.where(src_df['Year'] <= 2010).describe())

            Price        Year      Mileage
count 143274.0000 143274.0000  122931.0000
mean   10920.9394   2006.9331  112431.4654
std     9615.6580      2.7149   47737.7481
min        0.0000   1997.0000       5.0000
25%     5999.2500   2006.0000   82909.5000
50%     9299.0000   2007.0000  107684.0000
75%    13980.0000   2009.0000  136679.5000
max   490000.0000   2010.0000 2856196.0000


Пока только ясно, что древних машин в наборе мало. Попробую оценить выбросы по пробегу.

In [7]:
print(numeric.where(src_df['Mileage'] >= 1000000).describe())

           Price      Year      Mileage
count    12.0000   12.0000      12.0000
mean   8907.3333 2006.5833 1680738.5000
std    9008.4631    3.1176  555704.0516
min       0.0000 2002.0000 1000000.0000
25%    3899.7500 2005.0000 1392331.0000
50%    5490.5000 2006.0000 1511505.5000
75%   14013.2500 2008.0000 1895310.2500
max   31500.0000 2013.0000 2856196.0000


In [8]:
print(numeric.where(src_df['Mileage'] >= 500000).describe())

           Price      Year      Mileage
count    47.0000   47.0000      47.0000
mean  15533.0851 2007.2766 1066293.6596
std   18357.8012    4.8486  476885.9661
min       0.0000 1997.0000  527411.0000
25%    3449.5000 2004.5000  777427.0000
50%    5999.0000 2006.0000  999999.0000
75%   25835.0000 2011.0000  999999.5000
max   99999.0000 2017.0000 2856196.0000


In [9]:
print(numeric.where(src_df['Mileage'] >= 200000).describe())

           Price      Year      Mileage
count  4838.0000 4838.0000    4838.0000
mean   7299.2433 2004.5043  241234.6561
std    6862.7746    3.6458  100097.1164
min       0.0000 1997.0000  200000.0000
25%    3498.2500 2002.0000  209619.2500
50%    5500.0000 2004.0000  223245.5000
75%    9599.7500 2007.0000  246694.0000
max   99999.0000 2017.0000 2856196.0000


**Дилема.** Всё было бы понятно, если бы не 100+К значений NaN пробега. Нужно проанализировать какие авто попали в пробег более 200К.

А сколько моделей есть в наборе до 2004 и нет в после?

In [10]:
src_df['Model'] = proceed_cat_str(src_df['Model'])

before2004 = src_df.where(src_df['Year'] <= 2004)['Model'].value_counts().keys()
after2004 = src_df.where(src_df['Year'] > 2004)['Model'].value_counts().keys()

inter = np.setdiff1d(before2004, after2004)
print("There is {} unique models before 2004.".format(len(inter)))

There is 269 unique models before 2004.


И сколько записей входит в это условие?

In [11]:
old_model_df = src_df.query('Model in @inter')
print("There is {} rows to delete.".format(old_model_df.shape[0]))

There is 2765 rows to delete.


Итак, попробуем удалить записи, где: <br>
1. Уникальные модели до 2004-го года.
2. До 2004-го года выпуска.
3. Пробег более 200К

In [12]:
src_row_c = src_df.shape[0]

src_df = src_df.loc[
        (~src_df['Model'].isin(old_model_df['Model'])) &
        (src_df['Year'] > 2004) &
        (src_df['Mileage'] < 200000)
]

processed_row_c = src_df.shape[0]

print("{} rows was deleted.".format(src_row_c - processed_row_c))

144512 rows was deleted.


Интересно, а сколько порченых записей осталось.

In [13]:
print(src_df.isna().sum())

Price          0
Year           0
Mileage        0
City           0
State          0
Vin            0
Make           0
Model      64071
dtype: int64


Вот это номер - ушли все порченные значения пробега!

**Рассуждение.**

На первый взгляд, то к чему я пришел выглядит не обоснованным. Шутка ли, 140К записей убрать? <br>
Но на самом деле, всё логично. 
1. США автомобильная страна. Автомобили стоят там дешево. Для штатов автомобиль старше 15 лет - металлолом. Или коллекционная модель. Поэтому при заполнении базы для таких авто могли просто пробег не указывать - зачем?
2. Автомобили с большим пробегом вполне могут стоить денег - грузовики и коммерческий ресурсный транспорт. И вот вопрос - а он нам в модели нужен? Мы же явно работаем с легковыми автомобилями.
3. Рассуждение касательно коллекционных автомобилей то-же, что для коммерческих.

In [14]:
print(src_df.loc[src_df['Price'] == 0].shape[0])

44210


Как видно, нам ещё есть что предсказывать.

![title](img/09.png)

Намного более приятная картина.

Хотя явно видно, что за какую-то модель золотишка отсыпают вне зависимости от пробега. Уж не NaN ли это? Узнаем истину вместе, как говорится.

# **Категориальные данные.**

В общем случае есть два варианта, закодировать числом или унитарным кодом. Я экспериментировал с унитарным кодом - мне очень не понравилось. Если категорий мало, до десятка, то еще ничего. 
Но при большом количестве категорий унитарный код создает слишком большую матрицу, что приводит к катострофическому расходу памяти. Поэтому я буду использовать Label encodng. 

In [15]:
src_df.drop('Vin', inplace=True, axis=1)

src_df['Make'] = proceed_cat_str(src_df['Make'])
src_df['City'] = proceed_cat_str(src_df['City'])
src_df['State'] = proceed_cat_str(src_df['State'])

temp_df = src_df[['Model', 'Make', 'City', 'State']].astype('category').apply(lambda x: x.cat.codes)
temp_df = temp_df.where(~src_df.isna(), src_df)

src_df['Model'] = temp_df['Model'].astype('float')
src_df['Make'] = temp_df['Make']
src_df['City'] = temp_df['City']
src_df['State'] = temp_df['State']

del temp_df
gc.collect()

print(src_df.head)

<bound method NDFrame.head of         Price  Year    Mileage  City  State  Make     Model
0        8995  2014 35725.0000   626     43     0 1045.0000
2        8995  2013 48851.0000   626     43     0 1045.0000
3       10999  2014 39922.0000  2334      5     0 1044.0000
6       14490  2014 34032.0000   835     34     0 2030.0000
7       13995  2013 32384.0000  2280     44     0 2026.0000
...       ...   ...        ...   ...    ...   ...       ...
852117  63215  2017     9.0000   477      4    53 1756.0000
852118  72260  2017  3201.0000   636     31    53 2216.0000
852119  55999  2016 28941.0000   717      5    53 2210.0000
852120  60240  2017  3005.0000  1875      4    53 2069.0000
852121  76995  2017  2502.0000  1484     34    53 2217.0000

[707593 rows x 7 columns]>


In [17]:
print(src_df.isna().sum())

Price          0
Year           0
Mileage        0
City           0
State          0
Make           0
Model      64071
dtype: int64


![title](img/10.png)

![title](img/11.png)

На данный момент никаких аномалий я не наблюдаю.

# **Импутация данных.**

Собственно, нужно импутировать только модель.

Посмотрим, где нулевая цена пересекается с порченной моделью.

In [16]:
temp_df = src_df.loc[
    (src_df['Price'] == 0) &
    (src_df['Model'].isna())
]

print(temp_df.shape[0])

4037


По крайней мере 90% моделей можно попробовать предсказать.